In [2]:
# Import relevant libaries
%matplotlib qt5 

import mne
import os
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs, create_ecg_epochs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# We store significant sensors to this variable so don't remove 
sign_sensors = []

In [4]:
vhdr_files = ['382733.vhdr', '382733_2.vhdr', '477819.vhdr', '477819_2.vhdr', '554432.vhdr', '554432_2.vhdr', '884723.vhdr', '884723_2.vhdr', '928376.vhdr', '928376_2.vhdr', '990291.vhdr', '990291_2.vhdr']
print(vhdr_files)
temp_list = vhdr_files[0:2]
print(temp_list)

['382733.vhdr', '382733_2.vhdr', '477819.vhdr', '477819_2.vhdr', '554432.vhdr', '554432_2.vhdr', '884723.vhdr', '884723_2.vhdr', '928376.vhdr', '928376_2.vhdr', '990291.vhdr', '990291_2.vhdr']
['382733.vhdr', '382733_2.vhdr']


In [5]:
filename = temp_list[0]
print(filename)

if filename[6] == '.':
    epochs_EN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:6] + '-epo.fif')
else:
    epochs_EN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:8] + '-epo.fif')
print(epochs_EN)

382733.vhdr
Reading data/epochs/processed_epochs/382733-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     700.00 ms
        0 CTF compensation matrices available
120 matching events found
Applying baseline correction (mode: mean)
120 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
<EpochsFIF  |   120 events (all good), -0.2 - 0.7 sec, baseline [-0.2, 0], ~26.5 MB, data loaded,
 'Dislike Conf': 2
 'Favourite Conf': 1
 'Favourite Match': 1
 'Finnish Conf': 24
 'Finnish Match': 25
 'Foreing Conf': 22
 'Foreing Match': 27
 'Neutrals': 10
 'Unknowns': 8>


In [6]:
filename = temp_list[1]
print(filename)

if filename[6] == '.':
    epochs_FIN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:6] + '-epo.fif')
else:
    epochs_FIN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:8] + '-epo.fif')
print(epochs_FIN)

382733_2.vhdr
Reading data/epochs/processed_epochs/382733_2-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     700.00 ms
        0 CTF compensation matrices available
120 matching events found
Applying baseline correction (mode: mean)
120 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
<EpochsFIF  |   120 events (all good), -0.2 - 0.7 sec, baseline [-0.2, 0], ~26.5 MB, data loaded,
 'Dislike Conf': 1
 'Dislike Match': 1
 'Favourite Conf': 1
 'Favourite Match': 1
 'Finnish Conf': 23
 'Finnish Match': 27
 'Foreing Conf': 25
 'Foreing Match': 23
 'Neutrals': 10
 'Unknowns': 8>


In [7]:
# Lets study Finnish Match / Conf and Foreing Match / Conf
finmatc = epochs_EN['Finnish Match'].average()
finconf = epochs_EN['Finnish Conf'].average()
formatc = epochs_EN['Foreing Match'].average()
forconf = epochs_EN['Foreing Conf'].average()

all_evokeds_EN = [finmatc, finconf, formatc, forconf]

finmatc = epochs_EN['Finnish Match']
finconf = epochs_EN['Finnish Conf']
formatc = epochs_EN['Foreing Match']
forconf = epochs_EN['Foreing Conf']
all_epochs_EN = [finmatc, finconf, formatc, forconf]
print(all_evokeds_EN)

# Lets study Finnish Match / Conf and Foreing Match / Conf
finmatc = epochs_FIN['Finnish Match'].average()
finconf = epochs_FIN['Finnish Conf'].average()
formatc = epochs_FIN['Foreing Match'].average()
forconf = epochs_FIN['Foreing Conf'].average()

all_evokeds_FIN = [finmatc, finconf, formatc, forconf]

finmatc = epochs_FIN['Finnish Match']
finconf = epochs_FIN['Finnish Conf']
formatc = epochs_FIN['Foreing Match']
forconf = epochs_FIN['Foreing Conf']
all_epochs_FIN = [finmatc, finconf, formatc, forconf]
print(all_evokeds_FIN)

[<Evoked  |  'Finnish Match' (mean, N=25), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Finnish Conf' (mean, N=24), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Match' (mean, N=27), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Conf' (mean, N=22), [-0.2, 0.7] sec, 32 ch, ~311 kB>]
[<Evoked  |  'Finnish Match' (mean, N=27), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Finnish Conf' (mean, N=23), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Match' (mean, N=23), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Conf' (mean, N=25), [-0.2, 0.7] sec, 32 ch, ~311 kB>]


In [8]:
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.utils import resample
## The statistical analysis ###
# WRITTEN: Ali Salloum 4.5.2019
# Modified: Juho Aaltonen 5.5.2019

# We compare the Global field power distributions

FMAT_F = all_evokeds_FIN[0]
FCON_F = all_evokeds_FIN[1]
EMAT_F = all_evokeds_FIN[2]
ECON_F = all_evokeds_FIN[3]

FMAT_E = all_evokeds_EN[0]
FCON_E = all_evokeds_EN[1]
EMAT_E = all_evokeds_EN[2]
ECON_E = all_evokeds_EN[3]

all_evokeds_list = [FMAT_F, FCON_F, EMAT_F, ECON_F, FMAT_E, FCON_E, EMAT_E, ECON_E]

# Preparing stuff
gfp_list = []
for term in all_evokeds_list:
    temp = np.sum(term.data ** 2, axis=0)
    gfp_list.append(mne.baseline.rescale(temp, term.times * 1e3, baseline=(None, 0)))


# Crosscorrelation between two signals, should be very low
cc = np.correlate(gfp_list[0], gfp_list[4], mode="valid")
print(cc)
# Perform the Kolmogorov-Smirnov test for goodness of fit, skip this now

plt.plot(all_evokeds_list[0].times * 1e3, gfp_list[0])
plt.plot(all_evokeds_list[4].times * 1e3, gfp_list[4])
plt.plot(all_evokeds_list[0].times * 1e3, gfp_list[1])
plt.plot(all_evokeds_list[4].times * 1e3, gfp_list[5])
plt.legend(('Finnish FMAT', 'English FMAT', 'Finnish FCON', 'English FCON'))

plt.show()

Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
[8.23242863e-17]


In [9]:
# Peforming the bootstrapping, I spoke about this method in our last meeting

gfp_bootstrap = []
for j in range(0,8):
    gfp_bootstrap.append([])
    for i in range(10000):
        np.random.seed(i)
        gfp_bootstrap[j].append((resample(gfp_list[j])))
    gfp_bootstrap[j] = np.mean(gfp_bootstrap[j], axis=1)
print('Bootstrapping finished!')

Bootstrapping finished!


In [10]:
# Plot bootstraps
plt.hist(gfp_bootstrap[0],alpha = 0.5)
plt.hist(gfp_bootstrap[1],alpha = 0.5)
plt.hist(gfp_bootstrap[2],alpha = 0.5)
plt.hist(gfp_bootstrap[3],alpha = 0.5)
plt.hist(gfp_bootstrap[4],alpha = 0.5)
plt.hist(gfp_bootstrap[5],alpha = 0.5)
plt.hist(gfp_bootstrap[6],alpha = 0.5)
plt.hist(gfp_bootstrap[7],alpha = 0.5)
#plt.legend(('FMAT_F', 'FCON_F', 'EMAT_F', 'ECON_F', 'FMAT_E', 'FCON_E', 'EMAT_E', 'ECON_E'))
plt.legend(('Finnish FMAT', 'Finnish FCON', 'Finnish FORMAT', 'Finnish FORCON', 'English FMAT', 'English FCON', 'English FORMAT', 'English FORCON'))
plt.xlabel("Voltage (µV)")
plt.ylabel("Frequency")
plt.title('Bootstrap ' + filename[:6])

Text(0.5,1,'Bootstrap 382733')

In [ ]:
j = 0
dif_bootstrap_means = []
print('This may take a while')
for term1 in gfp_list:
    print('New term1 begins')
    dif_bootstrap_means.append([])
    k = 0
    for term2 in gfp_list:
        combined = np.concatenate((term1, term2), axis=0)
        perms_temp1 = []
        perms_temp2 = []
        if k == 3:
            print('Half point of this term!')
        for i in range(10000):
            np.random.seed(i)
            perms_temp1.append(resample(combined, n_samples = len(term1)))
            perms_temp2.append(resample(combined, n_samples = len(term2)))
            
        dif_bootstrap_means[j].append(np.mean(perms_temp1, axis=1)-np.mean(perms_temp2, axis=1))
        k += 1 
    j += 1
    print(j)
print('Bootstrap mean differences are now computed')

This may take a while
New term1 begins
Half point of this term!
1
New term1 begins
Half point of this term!
2
New term1 begins
Half point of this term!


In [ ]:
j = 0
obs_difs = []
p_value = []
for term1 in gfp_list:
    obs_difs.append([])
    p_value.append([])
    k = 0
    for term2 in gfp_list:
        obs_difs[j].append(np.mean(term1) - np.mean(term2))
        p_value[j].append(dif_bootstrap_means[j][k][dif_bootstrap_means[j][k] >= obs_difs[j][k]].shape[0]/10000)
        if j == k:
            print(dif_bootstrap_means[j][k])
            print(obs_difs[j][k])
            print(p_value[j][k])
        k += 1
    j += 1
print('observed differences and p-values are computed!')




In [ ]:
#print('P-values from bootstarping different test and stimuli sets. Subject: {:6s}'.format(filename[:6]))
print('{:8} {:8s} {:8s} {:8s} {:8s} {:8s} {:8s} {:8s} {:8s}'.format('', 'FMAT_F', 'FCON_F', 'EMAT_F', 'ECON_F', 'FMAT_E', 'FCON_E', 'EMAT_E', 'ECON_E'))
header = ['FMAT_F', 'FCON_F', 'EMAT_F', 'ECON_F', 'FMAT_E', 'FCON_E', 'EMAT_E', 'ECON_E']

for i in range(0,8):
    print('{:8s} {:<8.4f} {:<8.4f} {:<8.4f} {:<8.4f} {:<8.4f} {:<8.4f} {:<8.4f} {:<8.4f}'.format(header[i],
                                                                                                 p_value[i][0],p_value[i][1],p_value[i][2],p_value[i][3],
                                                                                                p_value[i][4],p_value[i][5],p_value[i][6],p_value[i][7]))
    print('')